In [1]:
import numpy
import scipy.stats
from scipy.integrate import quad

In [2]:
from timeit import default_timer as timer

In [3]:
LOW_BOUND = -10.
UPP_BOUND = 10.

In [4]:
N = 10
M = 2000000

# Gaussian expectation vector
Mu = numpy.zeros(N)

# Gaussian variance-covariance matrix
Sigma = numpy.array([[ 2.11,  0.37, -0.42, -0.2 , -1.73, -0.54,  0.42,  0.81, -0.2 , -0.94],
                     [ 0.37,  1.78, -0.45, -0.25, -1.73,  0.04,  0.35,  0.38, -0.91, -0.48],
                     [-0.42, -0.45,  0.87,  0.09,  0.4 , -0.05, -0.25,  0.06,  0.16,  0.45],
                     [-0.2 , -0.25,  0.09,  0.19,  0.38, -0.04, -0.11, -0.04,  0.22,  0.17],
                     [-1.73, -1.73,  0.4 ,  0.38,  5.3 ,  0.61,  0.46,  0.26,  1.74,  1.46],
                     [-0.54,  0.04, -0.05, -0.04,  0.61,  0.53,  0.1 , -0.3 , -0.1 ,  0.17],
                     [ 0.42,  0.35, -0.25, -0.11,  0.46,  0.1 ,  0.91,  0.51, -0.17, -0.25],
                     [ 0.81,  0.38,  0.06, -0.04,  0.26, -0.3 ,  0.51,  1.55,  0.49,  0.08],
                     [-0.2 , -0.91,  0.16,  0.22,  1.74, -0.1 , -0.17,  0.49,  1.33,  0.65],
                     [-0.94, -0.48,  0.45,  0.17,  1.46,  0.17, -0.25,  0.08,  0.65,  0.88]])

print "Mu = %s" % Mu
print "Sigma = \n %s" % Sigma

ts_rv = timer()
rv = scipy.stats.multivariate_normal (mean=Mu, cov=Sigma, allow_singular=True)
X = rv.rvs(size=M)
te_rv = timer()

ctp = te_rv - ts_rv

Mu = [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Sigma = 
 [[ 2.11  0.37 -0.42 -0.2  -1.73 -0.54  0.42  0.81 -0.2  -0.94]
 [ 0.37  1.78 -0.45 -0.25 -1.73  0.04  0.35  0.38 -0.91 -0.48]
 [-0.42 -0.45  0.87  0.09  0.4  -0.05 -0.25  0.06  0.16  0.45]
 [-0.2  -0.25  0.09  0.19  0.38 -0.04 -0.11 -0.04  0.22  0.17]
 [-1.73 -1.73  0.4   0.38  5.3   0.61  0.46  0.26  1.74  1.46]
 [-0.54  0.04 -0.05 -0.04  0.61  0.53  0.1  -0.3  -0.1   0.17]
 [ 0.42  0.35 -0.25 -0.11  0.46  0.1   0.91  0.51 -0.17 -0.25]
 [ 0.81  0.38  0.06 -0.04  0.26 -0.3   0.51  1.55  0.49  0.08]
 [-0.2  -0.91  0.16  0.22  1.74 -0.1  -0.17  0.49  1.33  0.65]
 [-0.94 -0.48  0.45  0.17  1.46  0.17 -0.25  0.08  0.65  0.88]]


# Link between paper and code

\begin{equation*}
\ell(x) = \sum_k x_k + \frac{1}{2} \sum_k (x_k^+)^2
\end{equation*}

We need to solve the following system:
    
\begin{equation*}
\begin{cases}
\lambda \left[ 1
+ \mathbb{E} \left( \left(X_k - m_k^* \right)^+ \right) 
\right] = 1 \text{ for all $k$} \\
\sum_k \mathbb{E} \left( X_k - m_k^* \right) 
+ \sum_k \frac{1}{2} \mathbb{E} \left( \left[ \left( X_k - m_k^* \right)^+ \right]^2 \right) 
= c
\end{cases}
\end{equation*}

Definitions from the paper:

\begin{equation*}
\begin{cases}
\lambda \left[ 1
+ f_k \left( m_k^* \right) 
\right] = 1 \text{ for all $k$} \\
\sum_k \left( e_k \left( m_k^* \right) 
+ \frac{1}{2} g_k \left( m_k^* \right)  \right) 
= c
\end{cases}
\end{equation*}

In [5]:
def e(index, x):
    mu = Mu[index]
    
    return mu - x

In [6]:
def f(k, x):
    global X
    Xk = X[:, k]
    Xkmx = Xk - x    
    
    Xkmxp = numpy.maximum(Xkmx, 0.)
    
    return numpy.mean(Xkmxp)

In [7]:
def g(k, x):
    global X
    Xk = X[:, k]
    Xkmx = Xk - x    
    
    Xkmxp = numpy.maximum(Xkmx, 0.)
    
    Xkmxp2 = numpy.square(Xkmxp)
    
    return numpy.mean(Xkmxp2)

In [8]:
class WrapperFunc(object):
    def __init__(self, func, indexes):
        self._i = indexes
        self._f = func
        
    def __call__(self, *x):
        tmp_args = self._i + list(x)
        return self._f(*tmp_args)

## Put the variables and functions to the remote engines

In [9]:
def init_funcs():
    funcs = dict()
    
    for k in range(N):
        ek = WrapperFunc(e, [k])
        fk = WrapperFunc(f, [k])
        gk = WrapperFunc(g, [k])
    
        funcs[k] = {"e": ek, "f": fk, "g": gk}
        
    return funcs

In [10]:
def prepare_parameterized_funcs(dict_funcs):
    res = dict()
        
    for k, v in dict_funcs.iteritems():
        for kk, vv in v.iteritems():
            if isinstance(vv, WrapperFunc):
                key = "%s%s" % (kk, k)
                res[key] = (vv, k)
        
    return res

In [11]:
def run_par_func(param):
    f = param[0]
    vm = []
    for i_m in param[1:]:
        vm.append(M[i_m])
    
    return f(*vm)

In [12]:
funcs = init_funcs()
flatten_funcs = prepare_parameterized_funcs(funcs)

In [13]:
from timeit import default_timer as timer

In [14]:
from scipy.optimize import fsolve

def optimize(c, **kwargs):
    global N
    
    __results = {key: dict() for key in "efg"}
        
    def fill_results(lbl, res):
        fname = lbl[0]
        k = int(lbl[-1])

        if len(lbl) == 3:
            j = int(lbl[1])
            if k not in __results[fname]:
                __results[fname][k] = dict()
                
            __results[fname][k][j] = res
        else:
            __results[fname][k] = res
    
    def objective(v_x):
        _lambda = v_x[-1]
        
        # Local version
        global M
        M = numpy.clip(v_x, LOW_BOUND, UPP_BOUND)
        f_res = map(run_par_func, flatten_funcs.values())
        
        for lbl, fx in zip(flatten_funcs.keys(), f_res):
            fill_results(lbl, fx)
            
        last = 0.
        res = numpy.zeros(N + 1)
        
        for k in range(N):
            res_k = 1 + __results["f"][k]
            res[k] = (_lambda * res_k - 1.)**2
            
            last += __results["e"][k] + .5 * __results["g"][k]
                    
        res[-1] = (last - c)**2
        
        #print M
        #print res
        #print
        
        return res
    
    guess = numpy.zeros(N + 1)
    if 'guess' in kwargs and kwargs['guess'] is not None:
        guess = kwargs.pop('guess')
        
    return fsolve(objective, guess, **kwargs) 

In [15]:
c = 1.

guess = numpy.zeros(N+1)

In [16]:
tic = timer()

continue_ = True
__i = 1
l_N = 0
while continue_:
    print "iteration nb %i" % __i
    optim_result = optimize(c, guess=guess, maxfev=1000, full_output=True)
    continue_ = optim_result[2] != 1
    if continue_:
        guess = optim_result[0]
        __i += 1
        l_N += optim_result[1]["nfev"]
        print
    
toc_m_tic = timer() - tic

print optim_result[-1]

iteration nb 1
The solution converged.


In [17]:
m_star = optim_result[0][:-1]
lb_star = optim_result[0][-1]
cto = toc_m_tic

print "Time of optimization: %.2f sec" % (cto)
print "m star = %s" % m_star
print "lambda star = %s" % lb_star

Time of optimization: 220.41 sec
m star = [ 0.41037629  0.29510399 -0.04957261 -0.34353733  1.34818729 -0.19337094
 -0.03396627  0.212968    0.13170787 -0.04472912]
lambda star = 0.715613921315


In [18]:
l_N += optim_result[1]["nfev"]

print "l(%i) = %i" % (N, l_N)
print "function evaluated at the output = %s" % optim_result[1]["fvec"]

l(10) = 165
function evaluated at the output = [  5.74494020e-24   1.65312075e-23   1.84075924e-20   1.72054780e-21
   3.64683460e-21   1.22273196e-23   6.63798103e-22   3.71288904e-23
   8.26735008e-22   6.98384834e-22   7.16937437e-21]


### Just need to copy the results in Excel:

In [19]:
print m_star
print
print "%f \n %i \n %.2f \n %.2f" % (lb_star, l_N, ctp, cto)

[ 0.41037629  0.29510399 -0.04957261 -0.34353733  1.34818729 -0.19337094
 -0.03396627  0.212968    0.13170787 -0.04472912]

0.715614 
 165 
 1.57 
 220.41
